# Scrap geo wiki page

In [1]:
import wikipedia
import mwclient
import threading
import pandas as pd
import os

In [2]:
NB_THREADS_BY_CAT = 40

In [3]:
wikipedia.set_lang("fr")
site = mwclient.Site('fr.wikipedia.org')

In [4]:
if not os.path.exists("data"):
  os.makedirs("data")

In [5]:
def scrap_page(title, i, results):
  try:
    wiki = wikipedia.page(title)
    summary = wiki.summary
    content = wiki.content
    entities = wiki.links
    results[i] = (title, summary, content, entities)
  except:
    pass

  #print(title)
  #print(summary)
  #print(content)
  #print(entities)

In [6]:
def scrap_pages(titles, first, last, results):
  for i in range(first, last):
    if i < len(titles):
      scrap_page(titles[i], i, results)

In [7]:
def scrap_category(site, category):  
  pages = site.search(category, namespace=0)
  titles = [page['title'] for page in pages]

  categories = [category] * len(titles)
  summaries = [None] * len(titles)
  contents = [None] * len(titles)
  entities = [None] * len(titles)
  
  results = [None] * len(titles)

  threads = []

  # Create threads
  step = len(titles) // NB_THREADS_BY_CAT
  i = 0
  while i < len(titles):
    thread = threading.Thread(target=scrap_pages, args=(titles, i, i + step, results))
    threads.append(thread)
    i += step

  # start threads
  for thread in threads:
    thread.start()

  # join
  for thread in threads:
    thread.join()

  # collect
  for i in range(len(results)):
    if results[i] is not None:
      titles[i], summaries[i], contents[i], entities[i] = results[i]

  return categories, titles, summaries, contents, entities

In [8]:
CATEGORIES = ["géographie générale", 'architecture urbanisme', 'géologie']

In [9]:
categories = []
titles = []
summaries = []
contents = []
entities = []

In [10]:
for category in CATEGORIES:
  cats, tits, sums, conts, entis = scrap_category(site, category)
  categories, titles, summaries, contents, entities
  
  categories = categories + cats
  titles = titles + tits
  summaries = summaries + sums
  contents = contents + conts
  entities = entities + entis

/home/raoufdine/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/raoufdine/anaconda3/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [11]:
df = pd.DataFrame({"categories": categories, "titles": titles, "summaries": summaries, "contents": contents, "entities": entities})

In [12]:
df.to_json("./data/wiki_geo.json")

In [13]:
df.head(10)

,categories,titles,summaries,contents,entities
0,géographie générale,Géographie,La géographie (du grec ancien γεωγραφία – geog...,La géographie (du grec ancien γεωγραφία – geog...,"[Alain Barré, Alphanumérique, American Geograp..."
1,géographie générale,Épistémologie de la géographie,Cet article traite de l'épistémologie de la gé...,Cet article traite de l'épistémologie de la gé...,"[Aménagement du territoire, Analyse spatiale, ..."
2,géographie générale,Coordonnées géographiques,Par coordonnées géographiques (ou encore « rep...,Par coordonnées géographiques (ou encore « rep...,"[100e méridien est, 100e méridien ouest, 101e ..."
3,géographie générale,Géographie physique,"La géographie physique, appelée aussi géograph...","La géographie physique, appelée aussi géograph...","[Afrique, Al Idrissi, Alexander von Humboldt, ..."
4,géographie générale,Géographie vidalienne,Les principes de la géographie vidalienne,Les principes de la géographie vidalienne\n\n\...,"[1903, Albert Demangeon, André Cholley, Annale..."
5,géographie générale,Position (géographie),"La position, en géographie est un point dans l...","La position, en géographie est un point dans l...","[Coordonnées cartésiennes, Coordonnées géograp..."
6,géographie générale,Marigot (géographie),Un marigot est une petite étendue d'eau fermée...,Un marigot est une petite étendue d'eau fermée...,"[Afrique subsaharienne, Amazonie, Bayou, Berna..."
7,géographie générale,Géographie de la Bretagne,Cet article décrit la géographie de la Bretagn...,Cet article décrit la géographie de la Bretagn...,"[1842, 1968, 1969, 1970, 1er janvier, 2005, 20..."
8,géographie générale,Géographie de Singapour,Singapour est une cité-État très urbanisée d'A...,Singapour est une cité-État très urbanisée d'A...,"[Altitude, Anglais, Années 1960, Années 2030, ..."
9,géographie générale,Géographie de l'Équateur,L'Équateur est un pays situé en Amérique du Su...,L'Équateur est un pays situé en Amérique du Su...,"[Amazone (fleuve), Amérique, Amérique du Sud, ..."


In [14]:
df.shape

(30000, 5)

In [15]:
df.isnull().sum()

categories      0
titles          0
summaries     785
contents      785
entities      785
dtype: int64

In [16]:
df.fillna(value=pd.np.nan, inplace=True)

/tmp/ipykernel_950170/4114817568.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.fillna(value=pd.np.nan, inplace=True)


In [17]:
df.dropna(inplace=True)

In [18]:
df.isnull().sum()

categories    0
titles        0
summaries     0
contents      0
entities      0
dtype: int64

In [19]:
df.shape

(29215, 5)